In [39]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score

In [23]:
dataTrain = pd.read_csv("extendedTrain.csv")
X_test = pd.read_csv("extendedTest.csv")

In [24]:
dataTrain = dataTrain.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
dTCols = dataTrain.columns.tolist()
y_train = dataTrain['TARGET']
X_train = dataTrain.drop('TARGET', axis=1)

In [25]:
X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [26]:
X_train.shape

(300780, 324)

In [27]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.8, stratify=y_train, random_state=42)

In [28]:
X_train.shape

(60156, 324)

In [29]:
X_valid.shape

(240624, 324)

XGBoost

In [30]:
#pip install xgboost

In [31]:
from xgboost import XGBClassifier
import xgboost as xgb


In [32]:
xgb_model = XGBClassifier(
    n_estimators = 1000,
    max_depth=5,
    learning_rate=0.1,
    objective='binary:logistic',
    gamma=0.1,
    subsample = 0.5,
)
#notes

#large gamma = more conservative 
#subsample = ratio of training instances, betw 0 and 1


In [33]:
xgb_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric='auc', early_stopping_rounds=10)

c:\Users\roman\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\roman\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.69410
[1]	validation_0-auc:0.70160
[2]	validation_0-auc:0.70474
[3]	validation_0-auc:0.70769
[4]	validation_0-auc:0.71171
[5]	validation_0-auc:0.71432
[6]	validation_0-auc:0.71682
[7]	validation_0-auc:0.71779
[8]	validation_0-auc:0.71958
[9]	validation_0-auc:0.72056
[10]	validation_0-auc:0.72125
[11]	validation_0-auc:0.72205
[12]	validation_0-auc:0.72267
[13]	validation_0-auc:0.72405
[14]	validation_0-auc:0.72494
[15]	validation_0-auc:0.72559
[16]	validation_0-auc:0.72706
[17]	validation_0-auc:0.72808
[18]	validation_0-auc:0.72841
[19]	validation_0-auc:0.72927
[20]	validation_0-auc:0.72995
[21]	validation_0-auc:0.73026
[22]	validation_0-auc:0.73065
[23]	validation_0-auc:0.73157
[24]	validation_0-auc:0.73192
[25]	validation_0-auc:0.73225
[26]	validation_0-auc:0.73316
[27]	validation_0-auc:0.73399
[28]	validation_0-auc:0.73575
[29]	validation_0-auc:0.73616
[30]	validation_0-auc:0.73643
[31]	validation_0-auc:0.73682
[32]	validation_0-auc:0.73719
[33]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [34]:
#predictions = xgb_model.predict(X_test)
predictions = xgb_model.predict_proba(X_test[X_train.columns])[:, 1]

In [41]:
cv = StratifiedKFold(n_splits = 3, shuffle=True, random_state=42)
precision = cross_val_score(xgb_model, X_valid, y_valid, cv=cv, scoring='precision')
recall = cross_val_score(xgb_model, X_valid, y_valid, cv=cv, scoring='recall')


In [42]:
print("Precision score:", precision)

Precision score: [0.4394993  0.4587156  0.41793313]


In [43]:
print("Recall score:", recall)

Recall score: [0.04844397 0.04599111 0.04215205]


In [14]:
output = pd.DataFrame({'SK_ID_CURR': X_test['SK_ID_CURR'].astype(int), 'TARGET': predictions})
output.to_csv('OUTPUT_XGB.csv', index=False)

In [15]:
output

,SK_ID_CURR,TARGET
0,100001,0.057214
1,100005,0.183008
2,100013,0.017594
3,100028,0.052732
4,100038,0.115715
...,...,...
48739,456221,0.027614
48740,456222,0.077456
48741,456223,0.058201
48742,456224,0.071554


Cross validation on XGBoost

In [16]:
#dtrain = xgb.DMatrix(data=X_train, label=y_train)

In [17]:
#parameters = {'max_depth' : 3, 
#              'objective' : 'binary:logistic',
#              'eval_metric' : 'roc_auc'}

In [18]:
#cv_scores_xgb = xgb.cv(dtrain=dtrain, params=parameters, nfold=5, num_boost_round=10, metrics={'roc_auc'}, seed=0)

In [19]:
#cv_scores_xgb